In [ ]:
# Online test of Context Relevant By Yi 'Evelyn' He

In [240]:
import numpy as np
import pandas as pd
import matplotlib
#matplotlib.style.use('ggplot')
from sklearn import preprocessing
from sklearn import feature_selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from pandas.tools.plotting import scatter_matrix

In [2]:
import os
os.chdir("E:\Dataset\ContextRelevant")

In [3]:
'''
1. Load and summarize the included data
'''
# load data
data = pd.read_excel('ds_test_final.xls')
# check amounts of variables and records
print data.shape
# we get a data frame of 2215*7

# check missing values
print np.sum(data.isnull())
# so we have no missing values in this data set

print data.dtypes
# all columns are numeric

# generate summary statistics
#print data.describe

(2215, 7)
Dependent_variable    0
Fitted_residuals      0
Fitted_Values         0
V6                    0
V23                   0
V34                   0
V76                   0
dtype: int64
Dependent_variable      int64
Fitted_residuals      float64
Fitted_Values         float64
V6                      int64
V23                   float64
V34                   float64
V76                     int64
dtype: object


In [4]:
'''
2. Determine a reasonable measure of model quality, 
and use the included data to compute and report this statistic.
3. Do you consider it a good model? Why or why not?
'''
# since the dependent variable is real valued
# we can use RMSE (root-mean-square error) to evaluete the model
evl_RMSE = (np.mean(data.Fitted_residuals ** 2))**0.5
# here we get the initial RMSE is 569.779
# Obviously the initial model is not good, since the RMSE is too large

In [5]:
'''
4. Discuss how well this model performs relative to a baseline model.
'''
# comparing with baseline model
# here we use use a central tendency measure as baseline, i.e.mean
# calculate baseline measurement
baseline_RMSE = (np.mean((data.Dependent_variable - np.mean(data.Dependent_variable)) ** 2))**0.5
# Here we get the RMSE of baseline model: 615.459
# therefore the initial model performs worse than baseline model
# since we get smaller RMSE with baseline model

In [6]:

'''
5. Examine the columns of predictor variables. 
Note any predictors that should be transformed, 
dropped or interacted to improve the model.
'''
# build a Random Forest model to identify important factors
# just use default parameters
df_indep = data.ix[:, 3:]
#first build a linear regression model with all predictors-non-scaling
lm_1 = linear_model.LinearRegression()
lm_1.fit(df_indep, data["Dependent_variable"])
# Calculate RMSE
print("Root mean squared error: %.4f"
      % (np.mean((lm_1.predict(df_indep) - data["Dependent_variable"]) ** 2))**0.5)

Root mean squared error: 613.8905


In [7]:
#then scale all predictors
new_df = data.drop(["Fitted_residuals", "Fitted_Values"], 1)
min_max_scaler = preprocessing.MinMaxScaler()
new_df[['V6','V23','V34','V76']] = min_max_scaler.fit_transform(new_df[['V6','V23','V34','V76']])
lm_2 = linear_model.LinearRegression()
lm_2.fit(new_df.ix[:, 1:], new_df["Dependent_variable"])

# Calculate RMSE
print("Root mean squared error: %.4f"
      % (np.mean((lm_2.predict(new_df.ix[:, 1:]) - data["Dependent_variable"]) ** 2))**0.5)

Root mean squared error: 613.8905


In [8]:
# build Random Forest model with non-scaling data
model_RF_1 = RandomForestRegressor()
model_RF_1.fit(df_indep, data["Dependent_variable"])

# Calculate RMSE
print("Root mean squared error: %.4f"
      % (np.mean((model_RF_1.predict(df_indep) - data["Dependent_variable"]) ** 2))**0.5)

Root mean squared error: 282.2146


In [9]:
# build Random Forest model with scaled data
model_RF_2 = RandomForestRegressor()
model_RF_2.fit(new_df.ix[:, 1:], data["Dependent_variable"])

# Calculate RMSE
print("Root mean squared error: %.4f"
      % (np.mean((model_RF_2.predict(new_df.ix[:, 1:]) - data["Dependent_variable"]) ** 2))**0.5)

Root mean squared error: 278.0227


In [10]:
poly = preprocessing.PolynomialFeatures(2)
poly_df = poly.fit_transform(new_df.ix[:, 1:]) 
new_feature_names = poly.get_feature_names(list(new_df.columns.values)[1:])
poly_df = pd.DataFrame(poly_df, columns = new_feature_names)

In [11]:
poly_df.head()

,1,V6,V23,V34,V76,V6^2,V6 V23,V6 V34,V6 V76,V23^2,V23 V34,V23 V76,V34^2,V34 V76,V76^2
0,1.0,0.000270,0.019041,0.023013,0.666667,7.294498e-08,0.000005,0.000006,0.000180,0.000363,0.000438,0.012694,0.000530,0.015342,0.444444
1,1.0,0.001794,0.057572,0.058229,0.666667,3.218077e-06,0.000103,0.000104,0.001196,0.003314,0.003352,0.038381,0.003391,0.038819,0.444444
2,1.0,0.002645,0.061828,0.071653,0.666667,6.994059e-06,0.000164,0.000189,0.001763,0.003823,0.004430,0.041219,0.005134,0.047768,0.444444
3,1.0,0.000910,0.258289,0.289226,0.666667,8.272469e-07,0.000235,0.000263,0.000606,0.066713,0.074704,0.172192,0.083652,0.192817,0.444444
4,1.0,0.000170,0.247088,0.511681,0.333333,2.875442e-08,0.000042,0.000087,0.000057,0.061052,0.126430,0.082363,0.261817,0.170560,0.111111


In [12]:
# build Random Forest model with polynomial features data
model_RF_3 = RandomForestRegressor()
model_RF_3.fit(poly_df, data["Dependent_variable"])

# Calculate RMSE
print("Root mean squared error: %.4f"
      % (np.mean((model_RF_3.predict(poly_df) - data["Dependent_variable"]) ** 2))**0.5)

Root mean squared error: 287.6523


In [13]:
#find most important features
importances = model_RF_3.feature_importances_

In [14]:
importances

array([ 0.        ,  0.07085874,  0.06392881,  0.05639525,  0.00213018,
        0.02967908,  0.12406462,  0.1202588 ,  0.10793247,  0.05923673,
        0.10003982,  0.09843667,  0.06869049,  0.09665102,  0.00169732])

In [15]:
poly_df_important = poly_df.drop(["1","V76","V76^2" ], 1)

In [16]:
# build Random Forest model with only important polynomial features data
model_RF_4 = RandomForestRegressor()
model_RF_4.fit(poly_df_important, data["Dependent_variable"])

# Calculate RMSE
print("Root mean squared error: %.4f"
      % (np.mean((model_RF_4.predict(poly_df_important) - data["Dependent_variable"]) ** 2))**0.5)

Root mean squared error: 277.2574


In [17]:
model_RF_4.feature_importances_

array([ 0.07386484,  0.06282827,  0.06494122,  0.03358854,  0.11405021,
        0.1066092 ,  0.09404784,  0.06336372,  0.120465  ,  0.11059827,
        0.05667948,  0.09896343])

In [18]:
poly_df_important.columns.values

array([u'V6', u'V23', u'V34', u'V6^2', u'V6 V23', u'V6 V34', u'V6 V76',
       u'V23^2', u'V23 V34', u'V23 V76', u'V34^2', u'V34 V76'], dtype=object)

In [19]:
poly_df_important_1 = poly_df_important.drop(["V6","V23","V34","V6^2","V23^2","V34^2" ], 1)

In [20]:
# build Random Forest model with only important polynomial features data
model_RF_5 = RandomForestRegressor()
model_RF_5.fit(poly_df_important_1, data["Dependent_variable"])

# Calculate RMSE
print("Root mean squared error: %.4f"
      % (np.mean((model_RF_5.predict(poly_df_important_1) - data["Dependent_variable"]) ** 2))**0.5)

Root mean squared error: 275.2794


In [24]:
#remove more not very important features
poly_df_important_2 = poly_df_important[["V6 V23", "V6 V34", "V23 V34","V23 V76"]]
# build Random Forest model with only important polynomial features data
model_RF_6 = RandomForestRegressor()
model_RF_6.fit(poly_df_important_2, data["Dependent_variable"])

# Calculate RMSE
print("Root mean squared error: %.4f"
      % (np.mean((model_RF_6.predict(poly_df_important_2) - data["Dependent_variable"]) ** 2))**0.5)

Root mean squared error: 278.1396


#start to implement feature selection using sklearn.feature_selection

In [28]:
#first generate high order terms
poly3 = preprocessing.PolynomialFeatures(3)
poly3_df = poly3.fit_transform(new_df.ix[:, 1:]) 
new_feature_names = poly3.get_feature_names(list(new_df.columns.values)[1:])
poly3_df = pd.DataFrame(poly3_df, columns = new_feature_names)

In [41]:
#method 1: VarianceThreshold
selector = feature_selection.VarianceThreshold()
poly4 = selector.fit_transform(poly3_df)
new_feature_list = selector.get_support(indices = True)

In [46]:
new_feature_list

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34], dtype=int64)

In [72]:
temp = poly3_df.iloc[:, list(new_feature_list)]

Since there is only first feature removed, we won't run RF model

In [73]:
#method 2: SelectKBest
selector1 = SelectKBest(feature_selection.mutual_info_regression, k = 15)
kbest = selector1.fit_transform(temp, data["Dependent_variable"])
new_feature_list = selector1.get_support(indices = True)

D:\ANACONDA\envs\py2\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, _DataConversionWarning)


In [74]:
new_feature_list

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14], dtype=int64)

In [75]:
temp1 = temp.iloc[:, list(new_feature_list)]

In [149]:
#run RF model
model_RF_7 = RandomForestRegressor(500)
model_RF_7.fit(temp1, data["Dependent_variable"])

# Calculate RMSE
print("Root mean squared error: %.4f"
      % (np.mean((model_RF_7.predict(temp1) - data["Dependent_variable"]) ** 2))**0.5)

Root mean squared error: 237.0164


mutual_info_regression has better performance than  f_regression in our data set

In [128]:
#method 3: SelectPercentile
selector2 = feature_selection.SelectPercentile(feature_selection. mutual_info_regression, percentile = 65)
percent = selector2.fit_transform(temp, data["Dependent_variable"])
new_feature_list = selector2.get_support(indices = True)
temp2 = temp.iloc[:, list(new_feature_list)]
#run RF model
model_RF_8 = RandomForestRegressor()
model_RF_8.fit(temp2, data["Dependent_variable"])

# Calculate RMSE
print("Root mean squared error: %.4f"
      % (np.mean((model_RF_8.predict(temp2) - data["Dependent_variable"]) ** 2))**0.5)

Root mean squared error: 274.8152


In [129]:
new_feature_list

array([ 1,  2,  3,  4,  5,  6,  7,  8, 10, 14], dtype=int64)

In [118]:
#method 4: SelectFpr
selector3 = feature_selection.SelectFpr()
fpr = selector3.fit_transform(temp, data["Dependent_variable"])
new_feature_list = selector3.get_support(indices = True)
temp3 = temp.iloc[:, list(new_feature_list)]
#run RF model
model_RF_9 = RandomForestRegressor()
model_RF_9.fit(temp3, data["Dependent_variable"])

# Calculate RMSE
print("Root mean squared error: %.4f"
      % (np.mean((model_RF_9.predict(temp3) - data["Dependent_variable"]) ** 2))**0.5)

Root mean squared error: 290.2701


In [117]:
new_feature_list

array([ 2,  6, 14], dtype=int64)

In [150]:
#method 5: SelectFdr
selector4 = feature_selection.SelectFdr()
fdr = selector4.fit_transform(temp, data["Dependent_variable"])
new_feature_list = selector4.get_support(indices = True)
temp4 = temp.iloc[:, list(new_feature_list)]
#run RF model
model_RF_10 = RandomForestRegressor(500)
model_RF_10.fit(temp4, data["Dependent_variable"])

# Calculate RMSE
print("Root mean squared error: %.4f"
      % (np.mean((model_RF_10.predict(temp4) - data["Dependent_variable"]) ** 2))**0.5)

Root mean squared error: 535.0995


In [148]:
new_feature_list

array([ 2,  6, 14], dtype=int64)

In [124]:
#method 6: SelectFwe
selector5 = feature_selection.SelectFwe()
fwe = selector5.fit_transform(temp, data["Dependent_variable"])
new_feature_list = selector5.get_support(indices = True)
temp5 = temp.iloc[:, list(new_feature_list)]
#run RF model
model_RF_11 = RandomForestRegressor()
model_RF_11.fit(temp5, data["Dependent_variable"])

# Calculate RMSE
print("Root mean squared error: %.4f"
      % (np.mean((model_RF_11.predict(temp5) - data["Dependent_variable"]) ** 2))**0.5)

Root mean squared error: 542.6732


In [125]:
new_feature_list

array([2, 6], dtype=int64)

In [151]:
#method 7: GenericUnivariateSelect
selector6 = feature_selection.GenericUnivariateSelect(mode='percentile')
gus = selector6.fit_transform(temp, data["Dependent_variable"])
new_feature_list = selector6.get_support(indices = True)
temp6 = temp.iloc[:, list(new_feature_list)]
#run RF model
model_RF_12 = RandomForestRegressor(500)
model_RF_12.fit(temp6, data["Dependent_variable"])

# Calculate RMSE
print("Root mean squared error: %.4f"
      % (np.mean((model_RF_12.predict(temp6) - data["Dependent_variable"]) ** 2))**0.5)

Root mean squared error: 557.6821


In [145]:
new_feature_list

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14], dtype=int64)

for GenericUnivariateSelect, the best mode is k_best. Other modes selected out very limited features, like 1,2 or 3

In [171]:
#method 8: Recursive feature elimination
rfe = feature_selection.RFE(estimator=model_RF_7)
fit = rfe.fit(temp1, data["Dependent_variable"])

In [179]:
ranking = rfe.support_
selected = rfe.n_features_
est = rfe.estimator_

In [183]:
ranking

array([False,  True, False, False,  True, False, False,  True, False,
        True, False, False,  True,  True,  True], dtype=bool)

In [210]:
type(ranking)

numpy.ndarray

In [181]:
selected

7

In [175]:
est

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=500, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [186]:
temp1.shape

(2215, 15)

In [219]:
df_sel = temp1.iloc[:,ranking]

In [221]:
temp1.head()

,1,V34,V6^2,V6 V23,V6 V34,V34 V76,V6^2 V76,V6 V23^2,V6 V23 V34,V6 V23 V76,V6 V34^2,V6 V34 V76,V6 V76^2,V23^3,V34 V76^2
0,1.0,0.023013,7.294498e-08,0.000005,0.000006,0.015342,4.862999e-08,9.792356e-08,1.183470e-07,0.000003,1.430301e-07,0.000004,0.000120,0.000007,0.010228
1,1.0,0.058229,3.218077e-06,0.000103,0.000104,0.038819,2.145385e-06,5.945880e-06,6.013738e-06,0.000069,6.082371e-06,0.000070,0.000797,0.000191,0.025879
2,1.0,0.071653,6.994059e-06,0.000164,0.000189,0.047768,4.662706e-06,1.010961e-05,1.171608e-05,0.000109,1.357782e-05,0.000126,0.001175,0.000236,0.031846
3,1.0,0.289226,8.272469e-07,0.000235,0.000263,0.192817,5.514979e-07,6.067752e-05,6.794538e-05,0.000157,7.608377e-05,0.000175,0.000404,0.017231,0.128545
4,1.0,0.511681,2.875442e-08,0.000042,0.000087,0.170560,9.584806e-09,1.035273e-05,2.143890e-05,0.000014,4.439665e-05,0.000029,0.000019,0.015085,0.056853


In [220]:
df_sel.head()

,V34,V6 V34,V6 V23^2,V6 V23 V76,V6 V76^2,V23^3,V34 V76^2
0,0.023013,0.000006,9.792356e-08,0.000003,0.000120,0.000007,0.010228
1,0.058229,0.000104,5.945880e-06,0.000069,0.000797,0.000191,0.025879
2,0.071653,0.000189,1.010961e-05,0.000109,0.001175,0.000236,0.031846
3,0.289226,0.000263,6.067752e-05,0.000157,0.000404,0.017231,0.128545
4,0.511681,0.000087,1.035273e-05,0.000014,0.000019,0.015085,0.056853


In [222]:
# Calculate RMSE
print("Root mean squared error: %.4f"
      % (np.mean((est.predict(df_sel) - data["Dependent_variable"]) ** 2))**0.5)

Root mean squared error: 236.7121


In [231]:
#method 9: RFE with cross-validation
rfecv = feature_selection.RFECV(estimator = model_RF_7, cv = 10) 
fit = rfecv.fit(temp1, data["Dependent_variable"])
ranking = rfecv.support_
selected = rfecv.n_features_
est = rfecv.estimator_

In [232]:
ranking

array([False,  True, False,  True,  True, False, False,  True, False,
        True,  True, False,  True,  True,  True], dtype=bool)

In [233]:
selected

9

In [234]:
est

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=500, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [235]:
df_sel = temp1.iloc[:,ranking]

In [236]:
df_sel.head()

,V34,V6 V23,V6 V34,V6 V23^2,V6 V23 V76,V6 V34^2,V6 V76^2,V23^3,V34 V76^2
0,0.023013,0.000005,0.000006,9.792356e-08,0.000003,1.430301e-07,0.000120,0.000007,0.010228
1,0.058229,0.000103,0.000104,5.945880e-06,0.000069,6.082371e-06,0.000797,0.000191,0.025879
2,0.071653,0.000164,0.000189,1.010961e-05,0.000109,1.357782e-05,0.001175,0.000236,0.031846
3,0.289226,0.000235,0.000263,6.067752e-05,0.000157,7.608377e-05,0.000404,0.017231,0.128545
4,0.511681,0.000042,0.000087,1.035273e-05,0.000014,4.439665e-05,0.000019,0.015085,0.056853


In [237]:
# Calculate RMSE
print("Root mean squared error: %.4f"
      % (np.mean((est.predict(df_sel) - data["Dependent_variable"]) ** 2))**0.5)

Root mean squared error: 237.5194


In [261]:
#method 10: SelectFromModel
# use the base estimator LassoCV
clf = LassoCV()
#Set a minimum threshold of 0.15
sfm = SelectFromModel(clf, threshold = .15)
sfm.fit(temp1, data["Dependent_variable"])
n_features = sfm.transform(temp1).shape[1]
new_feature_list = sfm.get_support(indices = True)
df_sfm = temp1.iloc[:, list(new_feature_list)]
# Calculate RMSE
clf.fit(df_sfm, data["Dependent_variable"])
print("Root mean squared error: %.4f"
      % (np.mean((clf.predict(df_sfm) - data["Dependent_variable"]) ** 2))**0.5)

Root mean squared error: 613.4947


In [262]:
new_feature_list

array([ 1,  4,  5, 13, 14], dtype=int64)

Using this method we get the same selected features with RFE

In [299]:
#method 7: GenericUnivariateSelect
selector6 = feature_selection.GenericUnivariateSelect(mode='percentile')
gus = selector6.fit_transform(temp1, data["Dependent_variable"])
new_feature_list = selector6.get_support(indices = True)
temp6 = temp1.iloc[:, list(new_feature_list)]
#run RF model
model_RF_12 = RandomForestRegressor(500)
model_RF_12.fit(temp6, data["Dependent_variable"])

# Calculate RMSE
print("Root mean squared error: %.4f"
      % (np.mean((model_RF_12.predict(temp6) - data["Dependent_variable"]) ** 2))**0.5)

Root mean squared error: 557.5962


In [302]:
# try to use pipeline for method 7
import sklearn.pipeline
select =feature_selection.GenericUnivariateSelect(mode='percentile')
rf = RandomForestRegressor(500)
steps = [("feature selection", select), ("lcv", rf)]
pipeline = sklearn.pipeline.Pipeline(steps)
pipeline.fit(temp1, data["Dependent_variable"])
prediction = pipeline.predict(temp1)
print("Root mean squared error: %.4f"
      % (np.mean((prediction - data["Dependent_variable"]) ** 2))**0.5)

Root mean squared error: 557.7262


In [303]:
sklearn.metrics.mean_squared_error(data["Dependent_variable"], prediction)

311058.49984512338

In [306]:
(311058.49984512338)**0.5

557.726187160979